In [ ]:
library(hh.analytics)
library(hh.snowflake)
library(fs)
load.packages()

In [ ]:
##  Already in Snowflake
df_snowflake_837p = querySnowflake("select * from edwprodhh.edi_837p_parser.response_files")
df_snowflake_837i = querySnowflake("select * from edwprodhh.edi_837i_parser.response_files")

In [ ]:
df_files_all =
    list.files(
        "J:\\IU_Health_Complex\\837_FILES_IN\\",
        full.names = TRUE,
        recursive = TRUE
    ) %>% 
    data.frame(
        file_path = .
    ) %>% 
    mutate(
        file_name = file_path %>% str_extract("[^/]*$")
    ) %>%
    # filter(
    #     file_name %>% str_detect(".*_p.*\\.837$")
    # ) %>% 
    filter(
        !file_name %in% df_snowflake_837p$file_name &
        !file_name %in% df_snowflake_837i$file_name
    )

In [ ]:
df_files_all %>%
    mutate(
        move = file_path %>% 
            map(
                .f = ~ fs::file_move(
                    path = .x,
                    new_path = .x %>% str_remove("/archive")
                )
            )
    )